In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
rutor_url = 'http://5.45.84.133/new/'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [8]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
37,Зарубежные фильмы,11 Май 23,Дьявол внутри / Iblis dalam Kandungan (2022) WEB-DL 1080p | L2,1.59 GB,42,11
38,Зарубежные фильмы,11 Май 23,Крик 6 / Scream VI (2023) WEB-DLRip-AVC от ExKinoRay | D,1.64 GB,49,16
39,Зарубежные фильмы,11 Май 23,Тысяча и один / A Thousand and One (2023) WEB-DLRip от MegaPeer | TVShows,1.46 GB,42,9
41,Зарубежные фильмы,11 Май 23,Крик 6 / Scream VI (2023) WEB-DLRip от MegaPeer | D,1.45 GB,61,29
43,Зарубежные фильмы,11 Май 23,Крик 6 / Scream VI (2023) WEB-DLRip от MegaPeer | D,2.18 GB,50,13
47,Зарубежные фильмы,11 Май 23,Человек-муравей и Оса: Квантомания / Ant-Man and the Wasp: Quantumania (2023) UHD BDRemux 2160p от селезень | 4K | HDR | D,49.79 GB,46,100
50,Зарубежные фильмы,10 Май 23,"Крик 6 / Scream VI (2023) WEB-DL-HEVC 2160p от селезень | 4K | SDR | D, P, A",15.69 GB,46,30
51,Зарубежные фильмы,10 Май 23,Крик 6 / Scream VI (2023) WEB-DLRip-AVC от DoMiNo & селезень | D,1.45 GB,63,17
54,Зарубежные фильмы,10 Май 23,"Крик 6 / Scream VI (2023) WEB-DL-HEVC 2160p от селезень | 4K | HDR | HDR10+ | Dolby Vision Profile 8 | D, P, A",15.09 GB,46,28
55,Зарубежные фильмы,10 Май 23,"Крик 6 / Scream VI (2023) WEB-DLRip 720p от DoMiNo & селезень | D, P, A",5.64 GB,62,38


In [7]:
id = [20]
my_df.loc[id, 'href']

KeyError: "None of [Int64Index([20], dtype='int64')] are in the [index]"